# Imports

In [41]:
#Importar librerías necesarias para desarrollar el análisis
import importlib
import warnings

import pandas as pd
import numpy as np
import sklearn 
from sklearn.metrics import mean_squared_error, auc, log_loss, roc_auc_score, accuracy_score, confusion_matrix, classification_report, f1_score, recall_score, precision_score, make_scorer
import random as python_random
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy.stats import ks_2samp
import scorecardpy as sc
#import openpyxl

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.feature_selection import SelectKBest, chi2, RFE, mutual_info_classif
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import make_pipeline, Pipeline 
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc, log_loss, roc_auc_score, accuracy_score, confusion_matrix, classification_report, f1_score, recall_score, precision_score, make_scorer

from sklearn.model_selection import KFold, ShuffleSplit, LeaveOneOut, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 

from xgboost import XGBClassifier
from xgboost import XGBRegressor
import xgboost

import sys
import notebook
import jupyterlab

import os
from pathlib import Path

In [42]:
from dotenv import load_dotenv, find_dotenv
# Carga el .env más cercano hacia arriba en el árbol
load_dotenv(find_dotenv())

# Raíz del proyecto = carpeta que contiene el .env
PROJECT_ROOT = Path(find_dotenv()).parent

# DATA_PATH puede ser relativo (./data) o absoluto; aquí lo resolvemos desde el root
DATA_PATH = (PROJECT_ROOT / os.environ.get("DATA_PATH", "data")).resolve()

In [43]:
import sys, platform
print(sys.executable)
print(platform.python_version())

/Users/cristian/Documents/GitHub/adeslas-interview/.venv/bin/python
3.13.7


In [44]:
pd.set_option("display.max_columns", None)   # muestra todas las columnas
pd.set_option("display.width", 0)            # usa el ancho de la celda de Jupyter
pd.set_option("display.max_colwidth", None)  # no truncar el contenido de celdas

# Env

In [45]:
PROJ_ROOT = Path.cwd().parent 
sys.path.insert(0, str(PROJ_ROOT / "src"))


# Verifica
print("Añadido a sys.path:", sys.path[0])

from utils import feature_selection as fs

Añadido a sys.path: /Users/cristian/Documents/GitHub/adeslas-interview/src


# Funciones

# Load data

In [46]:
DATA_PATH = Path(os.environ["DATA_PATH"])

In [47]:
# Anulaciones
file_tab = DATA_PATH / "df_muestra_vars_eng.csv"
df_muestra_vars_eng = pd.read_csv(file_tab)

print(df_muestra_vars_eng.shape)
df_muestra_vars_eng.head(3)

(47547, 420)


,socio_art_viaje_bolsos_uso_pers,socio_serv_carpinteria,socio_serv_peaje_vehic_pers,socio_otros_productos_medicos,socio_combustibles_liquidos_viv_ppal,socio_transporte_aereo,socio_promed_m2_inmuebles_trast_alq,socio_num_inmuebles_casa_alq_6hab,socio_seguros_viaje_y_equipaje,socio_num_inmuebles_casa_alq,socio_funicular_teleferico_telesilla,anul_antig_cliente,socio_total_motos_y_ciclomotores,socio_med_general_y_pediatria_ambu,anul_flag_gestion_multas,socio_ing_fam_activas_por_miembro,socio_total_otros_prod_cuid_pers,anul_antiguedad_vehiculo,socio_precio_m2_nave_comp,socio_pct_fam_ing_superior_a_2589,socio_pct_fam_ing_entre_2122_y_2589,socio_total_serv_dentales,socio_electricidad_viv_ppal,socio_total_serv_seguros,socio_num_inmuebles_nave_comp,socio_peluq_hombres_y_ninos,socio_tratamiento_estetica_pers,socio_alquileres_garaje_otras_viv,socio_ingresos_brutos_asist_social,socio_seguros_otras_viv,socio_roommates_total,socio_num_inmuebles_piso_compra_3ban,socio_densidad_poblacion,socio_ingresos_anual_netos_pers,socio_ayuda_neta_familia_e_hijos,socio_serv_cuidado_ninos,socio_num_inmuebles_piso_compra_4ban,anula,socio_otros_serv_protecc_social,socio_jubilado_pct,socio_int_div_y_ganancias_brutas,socio_gastos_comunitarios_otras_viv,socio_promed_m2_inmuebles_garaje_comp,socio_num_inmuebles_piso_compra_0ban,socio_otros_aparat_y_mat_terap,socio_transf_perc_oh_netas,socio_num_inmuebles_casa_alq_7hab,socio_transporte_mar,socio_num_inmuebles_piso_compra_2hab,socio_aparatos_electr_cuid_pers,socio_num_inmuebles_local_comp,socio_num_inmuebles_piso_alq,socio_carbon_viv_ppal,socio_total_joyeria_bisut_y_reloj,socio_single_adulto_pct,socio_articulos_funerarios,socio_seguros_viv_ppal,socio_num_inmuebles_casa_alq_4hab,socio_otros_serv_paramedic_ambu,socio_renta_neta_alquiler_propiedad,socio_int_div_y_ganancias_netas,socio_num_inmuebles_casa_compra_7hab,socio_renta_disp_hogar,socio_num_inmuebles_piso_compra_1hab,socio_pareja_jubilados_pct,socio_precio_inmuebles_suelo_comp,socio_prueba_embarazo_y_anticoncep,socio_combustibles_liquidos_otras_viv,socio_num_inmuebles_piso_compra_5hab,socio_productos_farmaceuticos,socio_total_compra_rep_y_acces_vehic,socio_aparcamientos_public_y_parquim,anul_siniestros_ultimo_anyo,socio_promed_m2_inmuebles_nave_comp,socio_pct_fam_ing_entre_1641_y_2122,socio_num_inmuebles_piso_alq_2ban,socio_num_inmuebles_piso_compra_8hab,socio_precio_m2_nave_alq,socio_num_inmuebles_piso_alq_3ban,socio_nido_multinuclear_pct,socio_num_inmuebles_casa_compra_8hab,socio_articulos_bebe,socio_total_prod_farma_y_aparat_terap,socio_suministro_agua_viv_ppal,socio_total_electr_gas_y_otros_comb,socio_total_remesas,socio_ingresos_netos_mes_p60,socio_otros_comb_solidos_otras_viv,socio_total_alq_imput_viv_ppal,socio_total_otros_serv_rel_viv,socio_precio_inmuebles_piso_alq,socio_precio_m2_trast_comp,socio_num_inmuebles_piso_alq_4ban,socio_renta_disp_hogar_incl_prest,socio_total_serv_medic_ambulat,socio_nido_lleno_jov_total,socio_promed_m2_inmuebles_suelo_alq,socio_num_inmuebles_casa_alq_0hab,socio_promed_m2_inmuebles_ofic_comp,socio_total_otros_efectos_pers,socio_num_inmuebles_casa_compra_6hab,socio_num_inmuebles_piso_compra_3hab,socio_total_gastos_familiares,socio_num_familias,socio_num_inmuebles_piso_compra_6banx,socio_precio_inmuebles_local_comp,socio_reparacion_aparat_y_mat_terap,socio_precio_inmuebles_ofic_alq,socio_gas_ciudad_y_natural_viv_ppal,socio_total_dinero_menores_resid,socio_num_inmuebles_casa_compra_9habx,socio_pct_fam_ing_inferior_a_1389,socio_num_inmuebles_casa_alq_6banx,anul_importe_siniestros,socio_ayuda_bruta_familia_e_hijos,socio_serv_cuidado_pers_depend,socio_total_otros_serv_financ,socio_total_electricidad,socio_ingresos_netos_mes_p35,socio_rep_aparat_electr_cuid_pers,socio_taxi_y_coche_con_conductor,socio_ingresos_netos_asist_social,socio_num_inmuebles_casa_compra_4ban,socio_num_inmuebles_casa_alq_2hab,socio_nido_multinuclear_total,socio_ingresos_mes_netos_fam,socio_materiales_mant_y_rep_viv,socio_total_gas,socio_pct_fam_ing_entre_138

# Variables

In [48]:
var_target = 'anula'
vars_key=['id_poliza']


vars_flag=['anul_flag_gestion_multas', # gestion_multas_
           'anul_flag_vehiculo_sustitucion', #Viene de garantia_ampliada_norm
           'anul_flag_tipo_pago_anual'] #Viene de tipo_pago_norm


vars_num_socio = [c for c in  df_muestra_vars_eng.columns if c.startswith("socio_")]
vars_num_anul = [c for c in  df_muestra_vars_eng.columns if c.startswith("anul_")]

vars_total=vars_key+[var_target]+vars_num_socio+vars_num_anul

assert len(vars_total)==len(df_muestra_vars_eng.columns), "Faltan variables"


# Split train-val-test

In [49]:
df_muestra_vars_eng[vars_total].head()
df_muestra_vars_eng.rename(columns={'anula': 'target'}, inplace=True)

if "anula" in vars_total:
    idx = vars_total.index("anula")
    vars_total[idx] = "target"


In [50]:

df = df_muestra_vars_eng.copy()
assert {"target", "id_poliza"} <= set(df.columns)

# --- 60/20/20 estratificado ---
df_train, df_temp = train_test_split(
    df, test_size=0.40, stratify=df["target"], random_state=42
)
df_test, df_val = train_test_split(
    df_temp, test_size=0.50, stratify=df_temp["target"], random_state=42
)

# Features: quitar id_poliza para entrenar
X_train = df_train.drop(columns=["target", "id_poliza"])
y_train = df_train[["target"]]   # guardas id_poliza para join si quieres

X_val   = df_val.drop(columns=["target", "id_poliza"])
y_val   = df_val[["target"]]

X_test  = df_test.drop(columns=["target", "id_poliza"])
y_test  = df_test[["target"]]

# Guardado simple, sin tocar índices
X_train.to_csv(DATA_PATH / "X_train.csv", index=False)
y_train.to_csv(DATA_PATH / "y_train.csv", index=False)

X_val.to_csv(DATA_PATH / "X_val.csv", index=False)
y_val.to_csv(DATA_PATH / "y_val.csv", index=False)

X_test.to_csv(DATA_PATH / "X_test.csv", index=False)
y_test.to_csv(DATA_PATH / "y_test.csv", index=False)


# Feature Selection

In [51]:
X_train = pd.read_csv(DATA_PATH / "X_train.csv")
y_train = pd.read_csv(DATA_PATH / "y_train.csv")["target"]
df_train = X_train.copy()
df_train["target"] = y_train

In [52]:
features = [c for c in X_train.columns if c not in ["id_poliza"]]  # por si quedó alguna key

In [53]:
# --- 3) Filtro IV + KS ---
selected_vars, df_metrics = fs.filter_vars(df_train, features, target="target")

In [54]:
print("Variables seleccionadas:", len(selected_vars))
display(df_metrics.sort_values(["iv", "ks"], ascending=False).head(50))

Variables seleccionadas: 22


,feature,iv,ks
407,anul_pvp_vehiculo,0.717246,0.371113
414,anul_log_anul_pvp_vehiculo,0.717246,0.371113
416,anul_ratio_prima_pvp,0.420384,0.288159
156,anul_polizas_auto,0.121893,0.144382
11,anul_antig_cliente,0.052481,0.097252
173,anul_prima,0.049642,0.085438
413,anul_log_anul_prima,0.049642,0.085438
404,anul_edad_cliente,0.042227,0.062888
29,socio_seguros_otras_viv,0.039466,0.047147
12,socio_total_motos_y_ciclomotores,0.036261,0.054734


In [55]:
# --- 4) Importancias XGBoost ---
df_top = fs.top_xgb_vars(df_train, selected_vars, target="target", top_n=50)
print(len(df_top))
display(df_top)

22


,feature,importance
0,anul_pvp_vehiculo,0.177201
1,anul_ratio_prima_pvp,0.062599
2,anul_polizas_auto,0.057662
3,anul_profesion_norm_woe,0.046948
4,anul_cia_procedencia_norm_woe,0.046280
5,anul_antig_cliente,0.045523
6,socio_num_inmuebles_piso_compra_2hab,0.045422
7,anul_edad_cliente,0.043622
8,socio_total_motos_y_ciclomotores,0.042375
9,socio_pareja_joven_total,0.042275


## Eliminar las de baja importancia

In [56]:
to_remove = ['anul_log_anul_pvp_vehiculo', 'anul_log_anul_prima']

# Opción 1: reasignando (simple y clara)
df_top = df_top.loc[~df_top['feature'].isin(to_remove)].reset_index(drop=True)

df_top.head(100)



,feature,importance
0,anul_pvp_vehiculo,0.177201
1,anul_ratio_prima_pvp,0.062599
2,anul_polizas_auto,0.057662
3,anul_profesion_norm_woe,0.046948
4,anul_cia_procedencia_norm_woe,0.046280
5,anul_antig_cliente,0.045523
6,socio_num_inmuebles_piso_compra_2hab,0.045422
7,anul_edad_cliente,0.043622
8,socio_total_motos_y_ciclomotores,0.042375
9,socio_pareja_joven_total,0.042275


In [57]:
selected_vars = df_top['feature']
selected_vars.to_csv(DATA_PATH / "selected_vars.csv", index=False)

In [58]:
selected_vars = pd.read_csv(DATA_PATH / "selected_vars.csv")["feature"]
selected_vars

0                        anul_pvp_vehiculo
1                     anul_ratio_prima_pvp
2                        anul_polizas_auto
3                  anul_profesion_norm_woe
4            anul_cia_procedencia_norm_woe
5                       anul_antig_cliente
6     socio_num_inmuebles_piso_compra_2hab
7                        anul_edad_cliente
8         socio_total_motos_y_ciclomotores
9                 socio_pareja_joven_total
10      socio_promed_m2_inmuebles_nave_alq
11              socio_nido_lleno_jov_total
12    socio_num_inmuebles_casa_compra_1ban
13                socio_serv_cuidado_ninos
14       socio_num_inmuebles_casa_alq_4hab
15               socio_pareja_adulta_total
16                   socio_serv_fontaneria
17                      socio_num_familias
18                              anul_prima
19                      socio_num_personas
Name: feature, dtype: object